In [33]:
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

homepage = "https://www.hugi.is/forsida/korkar/?sida="
domain = "https://www.hugi.is"

thread_links = []


# max pages for hugi.is/korkar is 17837

for page in tqdm(range(1, 50)):
    response = requests.get(homepage + str(page))
    soup = BeautifulSoup(response.text, "html.parser")

    # Find all links to threads
    list_of_threads = soup.find("table")
    links = list_of_threads.find_all("a", href=re.compile("/korkar/"))
    links = [domain + link["href"] for link in links]

    thread_links.extend(links)

print(len(thread_links), "threads found")
thread_links[:5]

100%|██████████| 49/49 [00:18<00:00,  2.68it/s]

1225 threads found


['https://www.hugi.is/tilveran/korkar/803807/hvernig-getur-madur-fengid-sprautur/',
 'https://www.hugi.is/skoli/korkar/803794/glosur-laxdaela/',
 'https://www.hugi.is/smasogur/korkar/803780/brot-ur-sogu/',
 'https://www.hugi.is/djammid/korkar/803744/vardandi-nytt-nafn-b5/',
 'https://www.hugi.is/popp/korkar/803702/smellir-a-islensku-ensku-og-donsku/']

In [34]:
from bs4 import BeautifulSoup, NavigableString, Tag
import json

class Thread:
    def __init__(self, url):
        self.url = url
        self.data = self.get_thread_data(url)

    def __repr__(self):
        return json.dumps(self.data, ensure_ascii=False)

    def clean_text(self, html=""):
        soup = BeautifulSoup(html, 'html.parser')
        text = ''
        for child in soup.find_all('div')[0]:
            if isinstance(child, NavigableString):
                text += str(child).strip()
            elif isinstance(child, Tag):
                if child.name != 'br':
                    text += child.text.strip()
                else:
                    text += '\n'

        # result = text.strip().split('\n')
        result = text.strip()
        return result

    def get_thread_data(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        title = soup.find(class_="articlecontainer").find("h1").text
        question = self.clean_text(str(soup.find("div", class_="articlebody")))
        responses = soup.find_all("div", class_="replycontent")
        # exlude the first response, which is the same as the question
        responses = [self.clean_text(str(response)) for response in responses]
        return {"title": title.strip(), "question": question, "responses": responses, "url": url}

In [35]:
import tqdm.auto as tqdm
import pandas as pd
import json

with open("hugi/data.json", "w") as f:
    f.write("[")

    for url in tqdm.tqdm(thread_links):
        try:
            data = Thread(url).data
            if url != thread_links[0]: f.write(",")
            f.write(json.dumps(data, ensure_ascii=False))

        except Exception as e:
            print(e)
            print("could not scrape:", url)
        
    f.write("]")


  0%|          | 0/1225 [00:00<?, ?it/s]

In [38]:
import json
import pandas as pd

with open("hugi/data.json", "r") as f:
    data = json.load(f)

df = pd.DataFrame(data)
df.drop(columns=["url", "title"], inplace=True)
df = df[df.question.apply(len) > 0]
df = df[df.responses.apply(len) > 0]

# amount of data
print(df.shape)

with open("hugi/cleaned_data.txt", "w") as f:
    for i, row in df.iterrows():
        f.write("###QUESTION:" + "\n" + row.question + "\n\n")
        f.write("###RESPONSE:" + "\n" + row.responses[0] + "\n\n")

(513, 2)
